In [11]:
import pandas as pd

#Read in the S&P500 data
sp500 = pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/^GSPC?period1=0&period2=9999999999&interval=1wk&events=history&includeAdjustedClose=true')

#Calculate the 12-period and 26-period exponential moving averages (EMAs)
sp500['ema52'] = sp500['Adj Close'].ewm(span=52, adjust=False).mean()
sp500['ema113'] = sp500['Adj Close'].ewm(span=113, adjust=False).mean()

#Calculate the MACD line as the difference between the 12-period EMA and the 26-period EMA
sp500['macd'] = sp500['ema52'] - sp500['ema113']

#Calculate the 9-period EMA of the MACD line to get the signal line
sp500['signal'] = sp500['macd'].ewm(span=39, adjust=False).mean()

#Calculate the MACD histogram as the difference between the MACD line and the signal line
sp500['macd_hist'] = sp500['macd'] - sp500['signal']

#Calculate the lagged MACD histogram
sp500['macd_hist_lag'] = sp500['macd_hist'].shift(1)

#Calculate the periods when the MACD histogram turns positive from negative
sp500['signal_cross'] = ((sp500['macd_hist_lag'] < 0) & (sp500['macd_hist'] > 0)).astype(int)

#Calculate the 4-week, 8-week, and 52-week returns when the MACD histogram turns positive from negative
for i in range(4, 53, 4):
    col_name = 'return_{}w_signal_cross'.format(i)
    sp500[col_name] = sp500['Adj Close'].pct_change(periods=i)
    sp500[col_name] = sp500[col_name].shift(-i) * sp500['signal_cross']

#Export the data to a CSV file
sp500.to_csv('MACD_M2W.csv', index=False)